# Comprehensive Guide To Approximate Nearest Neighbors Algorithms

## Exhaustive Search Usage

In [ ]:
# !pip install faiss-gpu
# !pip install faiss-cpu


In [8]:
from pathlib import Path

In [ ]:
# does not work/url does not exit
# import pickle
# import faiss
# def load_data():
#     with open('movies.pickle', 'rb') as f:
#         data = pickle.load(f)
#     return data
#     # return datadata = load_data()
# data = load_data()
# print (data)

## recrateing movie lesne directly


### Load and Preprocess the Movie Data
You need to prepare your movie data for encoding. For example, you may want to combine the title, description, and genres to form a rich text representation of each movie.

In [ ]:
# import pandas as pd

# Path to the movie metadata file
# movies_file_path = 'ml-100k/u.item'
movies_file_path = Path('./MovieLens/ml-100k/u.item')
# Define all 24 column names based on the dataset description
movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + \
                [f'genre_{i}' for i in range(19)]  # 19 genre columns



# Load the data with the correct number of columns
movies = pd.read_csv(movies_file_path, sep='|', encoding='latin-1', header=None, names=movie_columns)

# Display the first few rows
print(movies.head())


   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0        1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0        0   

   genre_2  genre_3  genre_4  ...  genre_9  genre_10  genre_11  genre_12  \
0        0        1        1  ...        0         0         0         0   


In [9]:
# import pandas as pd
# import pickle

# Load the MovieLens 100K ratings data
# ratings_file_path = 'ml-100k/u.data'  # Replace with the correct path to the 'u.data' file
ratings_file_path = Path('./MovieLens/ml-100k/u.data')

# The data does not have headers in the file, so we specify them
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_file_path, sep='\t', names=columns)

# Display first few rows of the dataset
print(ratings.head())

# Save the ratings data as a pickle file
with open('movielens_100k_ratings.pickle', 'wb') as f:
    pickle.dump(ratings, f)

print("MovieLens 100K ratings data saved to movielens_100k_ratings.pickle!")


   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596
MovieLens 100K ratings data saved to movielens_100k_ratings.pickle!


In [14]:
# Example: Create a text representation for each movie by combining title and genres
movies['text_representation'] = movies['title'] + ' ' + movies['release_date']

# Display the dataset with the text representation
print(movies[['movie_id', 'text_representation']].head())

   movie_id            text_representation
0         1   Toy Story (1995) 01-Jan-1995
1         2   GoldenEye (1995) 01-Jan-1995
2         3  Four Rooms (1995) 01-Jan-1995
3         4  Get Shorty (1995) 01-Jan-1995
4         5     Copycat (1995) 01-Jan-1995


### Use a Pretrained Model to Generate Embeddings
Now, we’ll use a pretrained transformer model to create semantic embeddings for the movie text data. The model will convert the text representation of each movie into a vector of fixed size, capturing its semantic meaning.

In [15]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model (e.g., 'all-MiniLM-L6-v2' is small and fast)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate semantic embeddings for the movie text representations
movie_descriptions = movies['text_representation'].tolist()
movie_embeddings = model.encode(movie_descriptions, show_progress_bar=True)

# The movie_embeddings is a matrix where each row is the vector for a movie
print(f"Shape of embeddings: {movie_embeddings.shape}")


c:\Users\Tiger\anaconda3\envs\p311Udacity1\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Tiger\anaconda3\envs\p311Udacity1\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Batches: 100%|██████████| 53/53 [00:02<00:00, 20.81it/s]


Shape of embeddings: (1682, 384)


### Save the Encoded Data
You can now save these embeddings for future use (e.g., for similarity search or as input to a recommendation model). You might also want to save the movie titles along with the embeddings to keep track of which vector corresponds to which movie.

In [16]:
import pickle

# Save the movie embeddings and their corresponding titles
with open('movie_embeddings.pickle', 'wb') as f:
    pickle.dump((movies['movie_id'].tolist(), movie_embeddings), f)

print("Movie embeddings saved to 'movie_embeddings.pickle'")


Movie embeddings saved to 'movie_embeddings.pickle'


### Perform Similarity Search (Optional)
If your goal is to perform a similarity search (e.g., find similar movies based on their semantic embeddings), you can use FAISS, a library that allows efficient similarity searches over large vector spaces.

In [17]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
movie_embeddings_np = np.array(movie_embeddings)

# Create a FAISS index
index = faiss.IndexFlatL2(movie_embeddings_np.shape[1])  # L2 distance (Euclidean)
index.add(movie_embeddings_np)  # Add embeddings to the index

# Search for the 5 most similar movies to the first movie in the dataset
query_embedding = movie_embeddings_np[0].reshape(1, -1)
distances, indices = index.search(query_embedding, 5)

# Print the most similar movies
for idx in indices[0]:
    print(f"Movie ID: {movies.iloc[idx]['movie_id']}, Title: {movies.iloc[idx]['title']}")


Movie ID: 1, Title: Toy Story (1995)
Movie ID: 1219, Title: Goofy Movie, A (1995)
Movie ID: 1470, Title: Gumby: The Movie (1995)
Movie ID: 772, Title: Kids (1995)
Movie ID: 93, Title: Welcome to the Dollhouse (1995)


In [23]:
text_representation_array = np.array(movies['text_representation'])

array([[-0.08029179,  0.05800571,  0.08467735, ...,  0.01737713,
         0.03208596,  0.09082718],
       [-0.0951554 ,  0.01224546,  0.01386145, ...,  0.01851496,
        -0.03020019,  0.02669655],
       [ 0.03325386,  0.01620201,  0.01582782, ...,  0.07960308,
        -0.1131446 , -0.01210991],
       ...,
       [-0.07109044,  0.04043393,  0.01658245, ...,  0.04077495,
        -0.05728028,  0.00712466],
       [-0.06940193,  0.02557337, -0.06849493, ...,  0.07956854,
        -0.10395558, -0.03993929],
       [-0.02086033,  0.03892694, -0.06994103, ..., -0.0374028 ,
         0.00198372, -0.03016665]], dtype=float32)

### copy paper

In [21]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        # self.lab
        self.labels = labels
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [24]:
movie_desc = np.array(movies['text_representation'])
index = ExactIndex(movie_embeddings_np, movie_desc)
index.build()

In [29]:
# index.query(data['vector'][0])
index.query(movie_embeddings_np[0].reshape(1, -1))
# movie_embeddings_np[0]

['Toy Story (1995) 01-Jan-1995',
 'Goofy Movie, A (1995) 01-Jan-1995',
 'Gumby: The Movie (1995) 01-Jan-1995',
 'Kids (1995) 01-Jan-1995',
 'Welcome to the Dollhouse (1995) 24-May-1996',
 'Little Princess, A (1995) 01-Jan-1995',
 'Candyman: Farewell to the Flesh (1995) 01-Jan-1995',
 'Castle Freak (1995) 01-Jan-1995',
 'Boys Life (1995) 01-Jan-1995',
 'Friday (1995) 01-Jan-1995']

### ANN implementaion

In [ ]:
# !pip install annoy


In [36]:
from annoy import AnnoyIndex


In [39]:
# # class AnnoyIndex():
# #     def __init__(self, vectors, labels):
# #         self.dimension = vectors.shape[1]
# #         self.vectors = vectors.astype('float32')
# #         self.labels = labels    
   
# #     def build(self, number_of_trees=5):
# #         self.index = annoy.AnnoyIndex(self.dimension)
# #         for i, vec in enumerate(self.vectors):
# #             self.index.add_item(i, vec.tolist())
# #         self.index.build(number_of_trees)
        
# #     def query(self, vector, k=10):
# #         indices = self.index.get_nns_by_vector(
# #               vector.tolist(), 
# #               k, 
# #               search_k=search_in_x_trees)                                           
# #         return [self.labels[i] for i in indices]

# from annoy import AnnoyIndex

# class AnnoyIndexWrapper():
#     def __init__(self, vectors, labels):
#         self.dimension = vectors.shape[1]
#         self.vectors = vectors.astype('float32')
#         self.labels = labels    
   
#     def build(self, number_of_trees=5):
#         self.index = AnnoyIndex(self.dimension)
#         for i, vec in enumerate(self.vectors):
#             self.index.add_item(i, vec.tolist())
#         self.index.build(number_of_trees)
        
#     def query(self, vector, k=10):
#         indices = self.index.get_nns_by_vector(
#             vector.tolist(), 
#             k, 
#             search_k=search_in_x_trees
#         )                                           
#         return [self.labels[i] for i in indices]


In [ ]:
# ann_index = AnnoyIndex(movie_embeddings_np, movie_desc)
# ann_index.build()

In [43]:
class AnnoyIndexWrapper():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    
    def build(self, number_of_trees=5):
        self.index = AnnoyIndex(self.dimension, 'angular')  # Added metric type
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
    
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(
            vector.tolist(), 
            k, 
            search_k=-1
        )
        return [self.labels[i] for i in indices]


In [44]:
ann_index = AnnoyIndexWrapper(movie_embeddings_np, movie_desc)
ann_index.build()


In [56]:
# index.query(data['vector'][0])
# ann_index.query(movie_embeddings_np[0].reshape(1, -1))
query_vector = movie_embeddings_np[0].reshape(-1) 
ann_index.query(query_vector)

['Toy Story (1995) 01-Jan-1995',
 'Goofy Movie, A (1995) 01-Jan-1995',
 'Gumby: The Movie (1995) 01-Jan-1995',
 'Little Princess, A (1995) 01-Jan-1995',
 'Castle Freak (1995) 01-Jan-1995',
 'Boys Life (1995) 01-Jan-1995',
 'Kicking and Screaming (1995) 01-Jan-1995',
 'Dangerous Minds (1995) 01-Jan-1995',
 'Ghost in the Shell (Kokaku kidotai) (1995) 12-Apr-1996',
 'Clueless (1995) 01-Jan-1995']

### Vector Encoding Using LSH

In [51]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [53]:
index_lsh = LSHIndex(movie_embeddings_np, movie_desc)
index_lsh.build()

In [57]:
# query_vector.reshape(1, -1)
movie_embeddings_np[0]
index.query(movie_embeddings_np[0].reshape(1, -1))

['Toy Story (1995) 01-Jan-1995',
 'Goofy Movie, A (1995) 01-Jan-1995',
 'Gumby: The Movie (1995) 01-Jan-1995',
 'Kids (1995) 01-Jan-1995',
 'Welcome to the Dollhouse (1995) 24-May-1996',
 'Little Princess, A (1995) 01-Jan-1995',
 'Candyman: Farewell to the Flesh (1995) 01-Jan-1995',
 'Castle Freak (1995) 01-Jan-1995',
 'Boys Life (1995) 01-Jan-1995',
 'Friday (1995) 01-Jan-1995']

### Vector Encoding Using Quantization

In [59]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [60]:
Quant_index = IVPQIndex(movie_embeddings_np, movie_desc)
Quant_index.build()

In [61]:
Quant_index.query(movie_embeddings_np[0].reshape(1, -1))

['Toy Story (1995) 01-Jan-1995',
 'Goofy Movie, A (1995) 01-Jan-1995',
 'French Twist (Gazon maudit) (1995) 01-Jan-1995',
 'In the Bleak Midwinter (1995) 23-Feb-1996',
 'Target (1995) 28-Feb-1996',
 'Frighteners, The (1996) 19-Jul-1996',
 'Strange Days (1995) 01-Jan-1995',
 'Gumby: The Movie (1995) 01-Jan-1995',
 'Castle Freak (1995) 01-Jan-1995',
 'Wings of Courage (1995) 01-Jan-1995']

### Hierarchical Navigable Small World Graphs

In [63]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [2]:
!pip install nmslib
# !conda install -c conda-forge nmslib 



  Using cached nmslib-2.1.1.tar.gz (188 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
  Using cached numpy-2.1.2-cp310-cp310-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.1.2-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Running setup.py clean for nmslib
Failed to build nmslib


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1099 lines of output]
      Dependence list: ['pybind11<2.6.2', 'psutil', "numpy>=1.10.0,<1.17 ; python_version=='2.7'", "numpy>=1.10.0 ; python_version>='3.5'"]
      C:\Users\Tiger\anaconda3\envs\Py310Stanford2\lib\site-packages\setuptools\dist.py:452: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
             

In [5]:
# !conda install -c conda-forge mamba

!conda install nmslib 
# mamba install package_name
# python3.11 -m pip install --upgrade pip
# !pip install numpy
# !pip install cmake
# !pip install nmslib

# !pip install --no-cache-dir --only-binary :all: nmslib
# !conda create -n py310env python=3.10 -y



^C


In [6]:
import nmslib

NSM_index = NMSLIBIndex(movie_embeddings_np, movie_desc)
NSM_index.build()

ModuleNotFoundError: No module named 'nmslib'

In [1]:
!python --version
# !conda install python=3.10
# pip install -r requirements.txt
# !conda config --set channel_priority flexible


Python 3.10.15


In [ ]:
NSM_index.query(movie_embeddings_np[0].reshape(1, -1))